In [1]:
import sys
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.constant import (
    DB_CONN_JJTG_DATA,
)
from datetime import datetime
from dateutil.relativedelta import relativedelta
# import matplotlib.pyplot as plt

sql='''
SELECT MONTH,TICKER_SYMBOL,RETAIN_VALUE_MONTN_AVG
FROM xyzq_retain_fund
WHERE month=202209
and TICKER_SYMBOL in 
('163409','163417','163418','010147','163406','012557','007119','011338','007802','011932','163402','010027','008969','009007')
order by TICKER_SYMBOL
'''
data=    DB_CONN_JJTG_DATA.exec_query(sql)
data = data.set_index("TICKER_SYMBOL")
data['weight']=data['RETAIN_VALUE_MONTN_AVG']/data['RETAIN_VALUE_MONTN_AVG'].sum()

sql=f'''
select ticker_symbol,END_DATE,RETURN_RATE_TO_PREV_DT
from fund_adj_nav
where TICKER_SYMBOL in
('163409','163417','163418','010147','163406','012557','007119','011338','007802','011932','163402','010027','008969','009007')
and END_DATE>=20220124
'''
value=    DB_CONN_JJTG_DATA.exec_query(sql)

value=value.pivot(index='END_DATE',columns='ticker_symbol',values='RETURN_RATE_TO_PREV_DT')
# value_change_pct=value/value.shift(1)
# value_change_pct=value_change_pct.fillna(1)
weight_net_value=(value.dot(data['weight'])/100 + 1).cumprod()
weight_net_value=pd.DataFrame(data=weight_net_value,index=value.index)
query_sql = """
SELECT
	TRADE_DT AS END_DATE,
	PORTFOLIO_NAV 
FROM
	portfolio_nav 
WHERE
	TRADE_DT >= '20220124'
    and portfolio_name = '知己优选-进取全明星'
"""
portfolio_nav = DB_CONN_JJTG_DATA.exec_query(query_sql).set_index("END_DATE")
portfolio_nav = portfolio_nav/portfolio_nav.iloc[0,0]
weight_net_value = weight_net_value.merge(portfolio_nav, left_index=True, right_index=True)
weight_net_value.to_excel('D:/前20.xlsx')